# Structured Output Prediction of Anti-Cancer Drug Activity [1/2]

Anas Atmani, Benoît Choffin, Domitille Coulomb, Paul Roujansky

### Data

**Input:**

- We consider 2305 distinct molecules. Each of them has several physico-chemical and geometric properties that enables to build similarities between all molecules through a kernel. We end up with the [2305 x 2305] Gram matrix of the Tanimoto kernel.

**Ouput:**

- We have a total of 59 cancer cell lines for which we would like to predict the effect of each molecule (active/inactive). This last information is provided in a [2305 x 59] "target" matrix.

- We also have external RNA-based data for each cancer cell line. By computing the [59 x 59] correlation matrix based on these features, we build a similarity graph between all cancer cell lines through a *maximum weight spanning tree* (MWST). As a quick note, the graph should not necesarrily be fully-connected, which should considerably reduce computation time. We also compare the maximum weight spanning tree with the "correlation threshold method" for which the graph is built by assuming a relationship between two cancers if the composite correlation between them is above a certain threshold.

### Modelling

Two approaches:

- perform prediction independently for each cancer cell line, through a standard classification algorithm such as SVM;
- take into account the similarities between the cancer cell lines and make use of this "structure" (goal of the article) through a MMCRF algorithm.

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.sparse.csgraph import minimum_spanning_tree
import time
import seaborn as sns
from sklearn.metrics import f1_score

## Data import

In [3]:
file_names = ['ncicancer_input_kernel.txt',
            'ncicancer_bin_targets.txt',
            'ncicancer_targets.txt',
            'ncicancer_cancerCL_corr.txt']

data = []

' We import each dataset and append it to the list "data" '
for file in file_names:
    try:
        data.append(np.loadtxt('data/data_clean/'+file))
        print('%s loaded.' %file)
    except:
        print('Error: %s not loaded.' %file)

ncicancer_input_kernel.txt loaded.
ncicancer_bin_targets.txt loaded.
ncicancer_targets.txt loaded.
ncicancer_cancerCL_corr.txt loaded.


In [4]:
' We define the variables '
X_gram = data[0]
Y_class = data[1]
Y_reg = data[2]
cancer_correls = data[3]

In [5]:
' We check the shape of each variable '
X_gram.shape, Y_class.shape, Y_reg.shape, cancer_correls.shape

((2305, 2305), (2305, 59), (2305, 59), (59, 59))

## A - Method 1 : SVM

In [6]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

In [7]:
class k_fold_CV():
    
    """
    Class for SVM algorithm which makes it easy to compute k-fold cross-validation on it.
    """
    
    def __init__(self, C, n_splits, n_strats = 10):
        '''
        - "C"         SVM hyperparameter
        - "n_splits"  number of folds of the k-fold cross validation (at least 2)
        - "n_strats"  depth of the stratification considered for the stratified k-fold. The variable is the number of
                      cell lines against which the molecule is active.
        '''
        self.C = C
        self.n_splits = n_splits
        self.n_strats = n_strats
        self.trained = False
        
    def fit_predict(self, X, y, verbose=True):
        
        self.n, self.m = y.shape
        ' we build "n_splits" folds '
        skf = StratifiedKFold(n_splits=self.n_splits)
        
        ' create temporary variable for the stratification purpose '
        listOfPercent = np.percentile(np.sum(y, axis = 1),list((np.arange(self.n_strats))*10))
        bins = np.digitize(np.sum(y, axis = 1), listOfPercent)
        
        model = SVC(C=self.C, kernel='precomputed')
        
        if verbose:
            print("Fold \t Computation time")
        
        k = 0
        for train, test in skf.split(X, bins):
            ' we perform k-fold cross validation '
            startTime = time.time()
            
            X_train, y_train = X[train[:, None],train], y[train]
            X_test, y_test = X[test[:, None], train], y[test]
            
            Y_preds_j = []
            
            for j in range(y.shape[1]):
                ' we train j distinct models for each cancer cell line '
                model.fit(X_train, y_train[:,j])
                ' we stack the results iteratively '
                Y_preds_j.append(model.predict(X_test))
            
            ' We stack the results obtained for each fold '
            if k==0:
                self.Y_preds = np.array(Y_preds_j).T
            else:
                self.Y_preds = np.concatenate((self.Y_preds, np.array(Y_preds_j).T))
        
            runTime = time.time() - startTime
            if verbose:
                print("%d/%d \t %d" %(k+1, self.n_splits, runTime))
            k += 1
        
        ' we calculate the classification error '
        self.accuracies = np.array([accuracy_score(self.Y_preds[:,i], y[:,i]) for i in range(self.Y_preds.shape[1])])
        self.f1_scores = np.array([f1_score(y[:,i], self.Y_preds[:,i]) for i in range(self.Y_preds.shape[1])])
        self.trained = True
        
    def results(self):
        
        if self.trained:
            print("Results for %d folds on the full dataset:" %self.n_splits)
            print("Average accuracy = %.2f%%" %(np.mean(self.accuracies)*100))
            print("Standard deviation (accuracy) = %.2f%%" %(np.std(self.accuracies)*100))
            print("-----------------------------------------------------------")
            print("Average F1 score = %.2f%%" % (np.mean(self.f1_scores)*100))
            print("Standard deviation (F1 score) = %.2f%%" %(np.std(self.f1_scores)*100))
            
        else:
            print("Not trained yet.")
                    

We test the SVM algorithm on the full dataset (aka No-Zero-Active) with 5-fold cross-validation:

In [8]:
model = k_fold_CV(C=100, n_splits=5)
model.fit_predict(X_gram, Y_class)
model.results()

Fold 	 Computation time
1/5 	 5
2/5 	 6
3/5 	 7
4/5 	 6
5/5 	 5
Results for 5 folds on the full dataset:
Average accuracy = 60.76%
Standard deviation (accuracy) = 4.07%
-----------------------------------------------------------
Average F1 score = 43.84%
Standard deviation (F1 score) = 6.52%


Test on the reduced version of the dataset (aka Middle Active):

In [9]:
' Create the reduced dataset Middle Active '
y_bis = Y_class.copy()
y_bis[y_bis < 0] = 0.

idxMiddleActive = ((np.sum(y_bis, axis = 1) > 10) & (np.sum(y_bis, axis = 1) < 49))

In [10]:
model = k_fold_CV(C=100, n_splits=5)
model.fit_predict(X_gram[np.ix_(idxMiddleActive,idxMiddleActive)], Y_class[idxMiddleActive])
model.results()

Fold 	 Computation time
1/5 	 0
2/5 	 0
3/5 	 0
4/5 	 0
5/5 	 0
Results for 5 folds on the full dataset:
Average accuracy = 58.30%
Standard deviation (accuracy) = 7.97%
-----------------------------------------------------------
Average F1 score = 46.03%
Standard deviation (F1 score) = 19.48%


#### Please see MMCRF notebook for the other model.